## Modificiones previo a la estimacion
Tras haber creado el indice de incertidumbre, es interesante observar como interactua con otras series. Se usara un modelo con 5 variables

* Precios de exportacion (var. % anual)
* Incertidumbre (pts.)
* Tipo de cambio nominal (var. % anual)
* Inversion privada real (var. % anual)
* Inflacion (%)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from api_bcrp import api_bcrp 

In [2]:
series=[
    "PN02522AQ", #Inversión privada real var. % anual (trimestral)
    "PN01701BM", #Precios de exportación Índice 2007=100 (mensual) 
    "PN01207PM", #Tipo de cambio PEN/US (mensual)
    "PN38705PM" #IPC Dic2021=100 (mensual)
]

series_trimestrales = series[0:1]
series_mensuales = series[1:5]

Importando mediante la funcion `api_bcrp`

In [3]:
trim = api_bcrp(series_trimestrales, "1999Q1", "2019Q4")

Has importado tu variable PN02522AQ! 



In [4]:
mens = api_bcrp(series_mensuales, "1999-01", "2019-12")

Has importado tu variable PN01701BM! 

Has importado tu variable PN01207PM! 

Has importado tu variable PN38705PM! 



Transformaremos las variables para tenerlas en trimestral y en variaciones anuales

In [5]:
trim = trim.rename(columns={
    "PN02522AQ": "Inversión privada"
})


mens = mens.rename(columns={
    "PN01701BM": "IPX",
    "PN01207PM": "Tipo de cambio",
    "PN38705PM": "IPC"
})

In [6]:
# Trimestralizando
mens = mens.resample('Q').mean()

# Variaciones anuales
mens["Depreciacion"] = (mens["Tipo de cambio"]/mens["Tipo de cambio"].shift(4) - 1)*100
mens["Inflacion"] = (mens["IPC"]/mens["IPC"].shift(4) - 1)*100
mens["IPX"] = (mens["IPX"]/mens["IPX"].shift(4) - 1)*100

# Filtrando desde el periodo 2003Q3
mens = mens[mens.index>="2000-08-31"]

trim.index = trim.index.str[3:] + trim.index.str[:2]
trim.sort_index(inplace=True)
trim = trim.iloc[2:].reset_index()[["Inversión privada"]]

Importamos las series de incertidumbre

In [7]:
vix = pd.read_excel("./Base de datos/vix.xlsx", sheet_name="Trimestral", index_col="Date")
pca = pd.read_excel("./Base de datos/PCA.xlsx", sheet_name="Trimestral", index_col="Date")

vix = vix[vix.index>="2000-08-31"].reset_index()
pca = pca[pca.index>="2000-08-31"].reset_index()
vix

,Date,VIX
0,2000-09-30,18.263750
1,2000-12-31,26.037786
2,2001-03-31,25.608990
3,2001-06-30,24.006189
4,2001-09-30,26.414421
...,...,...
85,2021-12-31,19.241356
86,2022-03-31,25.299372
87,2022-06-30,27.306405
88,2022-09-30,24.814752


Unimos las bases de datos

In [8]:
data = mens[["IPX", "Depreciacion", "Inflacion"]].reset_index()
data = data.rename(columns={"time": "Date"})

# Uniones
data = data.merge(vix).merge(pca)
data["Inversión privada"] = trim

# Setting fecha
data.set_index("Date", inplace=True)

data

,IPX,Depreciacion,Inflacion,VIX,PCA,Inversión privada
Date,,,,,,
2000-09-30,6.083613,3.359590,3.710886,18.263750,-0.803214,-6.970631
2000-12-31,-1.458325,1.056022,4.001129,26.037786,0.284309,-3.419365
2001-03-31,-2.838051,1.666440,3.683259,25.608990,0.922136,-10.848616
2001-06-30,-2.367960,2.120980,2.587177,24.006189,0.166506,-0.423988
2001-09-30,-4.549860,0.405307,1.465562,26.414421,0.417470,1.657060
...,...,...,...,...,...,...
2018-12-31,-5.890072,3.452416,2.066755,21.231460,-0.798546,1.140884
2019-03-31,-8.484282,2.611762,2.126855,16.430785,-1.279382,2.908110
2019-06-30,-5.393379,1.905102,2.537244,15.168955,-1.797518,5.603677


Finalmente, solo para que ambas variables sean comparables, se decidió modificar el índice VIX y el PCA para mantener una misma dimensión

In [9]:
data["VIX"] = ((data["VIX"] - data["VIX"].mean())/ data["VIX"].std())*10
data["PCA"] = ((data["PCA"] - data["PCA"].mean())/ data["PCA"].std())*10

In [10]:
data.to_excel("./Base de datos/data_f.xlsx")